In [74]:
import numpy as np
import pandas as pd
import os

In [117]:
data_directory = "../Data/Study/05/moving/"
subdirectory = "recalibrated_calibration/"
fileList = os.listdir(data_directory + subdirectory)

In [96]:
def clean_df(df):
    result = df.loc[(df['Movement'] != "start") & (df['Movement'] != "transition") & (df['EyeTrackingEnabled'] == True) & (df['EyeCalibrationValid'] == True) & (df['EyeTrackingEnabledAndValid'] == True) & (df['EyeTrackingDataValid'] == True) & (df['GazeInputSource'] == "Eyes")].copy()
    #result = result[result['Movement'] != "transition"]
    #result = result[result['EyeTrackingDataValid'] == 'TRUE']
    #result = result[result['GazeInputSource'] == 'Eyes']
    result.reset_index(inplace=True, drop=True)
    return result

In [97]:
def calculate_cosine_error(df):
    df['cosineError'] = df.apply(lambda row: np.degrees(np.arccos(np.dot(np.array((row['EyeGazeDirection.x'], row['EyeGazeDirection.y'], row['EyeGazeDirection.z'])), np.array((row['localTransform.position.x'], row['localTransform.position.y'], row['localTransform.position.z'])))/(np.linalg.norm(np.array((row['EyeGazeDirection.x'], row['EyeGazeDirection.y'], row['EyeGazeDirection.z']))) * np.linalg.norm(np.array((row['localTransform.position.x'], row['localTransform.position.y'], row['localTransform.position.z'])))))), axis=1)
    return df

In [98]:
def calculate_euclidean_error(df):
    #df['euclideanError'] = df.apply(lambda row: np.linalg.norm(np.array((row['GazeAngleX'], row['GazeAngleY']))-np.array((row['TargetAngleX'], row['TargetAngleY']))), axis=1)
    #df['recalibratedEuclideanError'] = df.apply(lambda row: np.linalg.norm(np.array((row['CalibratedGazeAngleX'], row['CalibratedGazeAngleY']))-np.array((row['TargetAngleX'], row['TargetAngleY']))), axis=1)
    df['euclideanError'] = np.sqrt((df.GazeAngleX-df.TargetAngleX) * (df.GazeAngleX-df.TargetAngleX) + (df.GazeAngleY-df.TargetAngleY) * (df.GazeAngleY-df.TargetAngleY))
    df['recalibratedEuclideanError'] = np.sqrt((df.CalibratedGazeAngleX-df.TargetAngleX) * (df.CalibratedGazeAngleX-df.TargetAngleX) + (df.CalibratedGazeAngleY-df.TargetAngleY) * (df.CalibratedGazeAngleY-df.TargetAngleY))
    return df

In [99]:
def calculate_errors(output_subdirectory):
    for file in fileList:
        if '.csv' in file:
            df = pd.read_csv(data_directory + subdirectory + file)
            error_df = calculate_euclidean_error(df)
            error_df = calculate_cosine_error(error_df)
            if not os.path.exists(os.path.join(data_directory, output_subdirectory)):
                os.mkdir(os.path.join(data_directory, output_subdirectory))
            error_df_output_path = data_directory + output_subdirectory + '/' + file
            error_df.to_csv(error_df_output_path)
            print(file)
            print("uncalibrated: " + str(error_df['euclideanError'].mean()))
            print("calibrated: " + str(error_df['recalibratedEuclideanError'].mean()))
            print("cosine: " + str(error_df['cosineError'].mean()))
            print("")

In [118]:
calculate_errors("error")

calibration_20230505152814_shifted0_recalibrated_calibration.csv
uncalibrated: 3.5218710931403967
calibrated: 1.3750222775786236
cosine: 3.4481490072282646

ssHeadConstrained_20230505152308_shifted26_moving_recalibrated_calibration.csv
uncalibrated: 3.9770166926757153
calibrated: 2.217910315986425
cosine: 3.9145337939709623

wsWalking_20230505153932_shifted26_moving_recalibrated_calibration.csv
uncalibrated: 52.71128124371812
calibrated: 52.415947859843236
cosine: 87.51748923983907

wsWalking_20230505152552_shifted0_moving_recalibrated_calibration.csv
uncalibrated: 65.19725697092312
calibrated: 64.44306225339558
cosine: 89.8910202753688

hallway_20230505153353_shifted0_moving_recalibrated_calibration.csv
uncalibrated: 23.680789090210453
calibrated: 21.406833787617614
cosine: 23.2139466553226

ssWalking_20230505151621_shifted0_moving_recalibrated_calibration.csv
uncalibrated: 58.25925487280699
calibrated: 58.82922955453636
cosine: 86.2426382704535

wsBodyConstrained_20230505151239_shift